# Défi Grosses Data 2018 - Réseau de neurones avec scikit-learn

In [40]:
import numpy as np
import pandas as pd
import datetime as dt
from os import listdir
import imp
import Annex
imp.reload(Annex)
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [43]:
X_train,X_test,Y_train,Y_test,scaler=Annex.get_data_raw(scale=True, 
                                                        add_dummies=True,
                                                        var_dummies=['insee','ddH10_rose4'],
                                                        TrainTestSplit=True,
                                                        sz_test=0.3,
                                                        impute_method='drop',
                                                        convert_month2int=True,
                                                        date_method='week_number')#'drop'

We are addressing your request.
Data has been imported. Size: (189280, 31)


C:\Users\Quentin\Documents\INSA\5GMM\Defi\Annex.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df.mois[df.mois=='mars']='3'
C:\Users\Quentin\Documents\INSA\5GMM\Defi\Annex.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df.mois[df.mois=='avril']='4'
C:\Users\Quentin\Documents\INSA\5GMM\Defi\Annex.py:145: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df.mois[df.mois=='mai']='5'
C:\Users\Quentin\Documents\INSA\5GMM\Defi\Annex.py:146: SettingWithCopyWarning: 
A value is tryin

Months converted to int.
Dummies added.
Date transformed in week number.
26528 data points deleted. 14.02 %
Train size: 113926, Test size: 48826
Data scaled


In [44]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
param_grid=[{"hidden_layer_sizes":list([(80,)])}]#(50,),(60,),(70,),(80,),,(120,)
nnet= GridSearchCV(MLPRegressor(max_iter=500),param_grid,cv=10,n_jobs=-1)
nnetOpt=nnet.fit(X_train, Y_train)
# paramètre optimal
print("Meilleur score = %f, Meilleur paramètre = %s" % (1. - nnetOpt.best_score_,nnetOpt.best_params_))

Meilleur score = 0.030421, Meilleur paramètre = {'hidden_layer_sizes': (80,)}


In [6]:
# prévision de l'échantillon test
Y_pred = nnetOpt.predict(X_test)
print("MSE =",mean_squared_error(Y_test,Y_pred))
print("R2 =",r2_score(Y_test,Y_pred))

MSE = 1.2202695299
R2 = 0.968743106225


In [24]:
mlp=MLPRegressor(max_iter=500,hidden_layer_sizes =(80,80))
nnetOpt=mlp.fit(X_train, Y_train)

### Sammy, optimize!

In [9]:
import Sammy
import pickle
import os
path='./results/Resultats.pickle'
if os.path.exists(path):
    Resultats=pickle.load(open(path, 'rb'))
else:
    Resultats=Sammy.Results()
Resultats.say_hello()

Welcome, I am Sammy Nimizz.
May I help you?


In [11]:
Resultats.optimize(X_train,Y_train,nb_epoch=2,method=MLPRegressor,goal='min',coef_selection=0.2)
f=open('./results/Resultats.pickle', 'wb')
pickle.dump(Resultats,f)
f.close()

I want to improve myself!
I am too young to be optimized, let me try some randomness first.

Method is new? True 
 <class 'sklearn.neural_network.multilayer_perceptron.MLPRegressor'> {'warm_start': False, 'verbose': False, 'validation_fraction': 0.1, 'tol': 0.0001, 'solver': 'adam', 'shuffle': True, 'random_state': None, 'power_t': 0.5, 'nesterovs_momentum': True, 'momentum': 0.9, 'max_iter': 200, 'learning_rate_init': 0.001, 'learning_rate': 'constant', 'hidden_layer_sizes': (50, 50, 50, 50), 'epsilon': 1e-08, 'early_stopping': False, 'beta_2': 0.999, 'beta_1': 0.9, 'batch_size': 'auto', 'alpha': 0.0001, 'activation': 'relu'}
Running MLPRegressor with parameters: {'warm_start': False, 'verbose': False, 'validation_fraction': 0.1, 'tol': 0.0001, 'solver': 'adam', 'shuffle': True, 'random_state': None, 'power_t': 0.5, 'nesterovs_momentum': True, 'momentum': 0.9, 'max_iter': 200, 'learning_rate_init': 0.001, 'learning_rate': 'constant', 'hidden_layer_sizes': (50, 50, 50, 50), 'epsilon': 

Score: 9.691e-01


Epoch 1
I selected the 1 best solutions I know.
Mutation!
Evaluation...
Method is new? False 
 <class 'sklearn.neural_network.multilayer_perceptron.MLPRegressor'> {'warm_start': False, 'verbose': False, 'validation_fraction': 0.1, 'tol': 0.0001, 'solver': 'adam', 'shuffle': True, 'random_state': None, 'power_t': 0.5, 'nesterovs_momentum': True, 'momentum': 0.9, 'max_iter': 200, 'learning_rate_init': 0.001, 'learning_rate': 'constant', 'hidden_layer_sizes': (150, 150, 150, 150), 'epsilon': 1e-08, 'early_stopping': False, 'beta_2': 0.999, 'beta_1': 0.9, 'batch_size': 'auto', 'alpha': 0.0001, 'activation': 'relu'}

Epoch 2
I selected the 1 best solutions I know.
Mutation!
Evaluation...
Method is new? True 
 <class 'sklearn.neural_network.multilayer_perceptron.MLPRegressor'> {'warm_start': False, 'verbose': False, 'validation_fraction': 0.1, 'tol': 0.0001, 'solver': 'adam', 'shuffle': True, 'random_state': None, 'power_t': 0.5, 'nesterovs_momentum': True, 'momentum': 0.9

NameError: name 'pickle' is not defined

In [ ]:
print(Annex.Results(Resultats.bests(nb=5)))